# 简介

## 贝叶斯决策论

<img align="left" src="img/贝叶斯决策论.png" width="350px">  

基本原理：  

如果 $p_1(x_1,x_2)>p_0(x_1,x_2)$，那么类别为1；如果 $p_1(x_1,x_2)<p_0(x_1,x_2)$，那么类别为0；如果相等可设为1，也可设为0。  

即：对每个样本 $x$，选择能使后验概率 $p(c|x)$ 最大的类别标记（$c$ 为类别）。

## 贝叶斯决策模型

$$p(c|x)=p(x|c)p(c)$$  

> 注：分母的 $p(x)$ 在每次计算中都一样，因此可以省略

在实际问题中，数据量较大，无法算出所有 $p(x|c)$，因此我们计算它的极大似然估计。  
设 $p(x|c)$ 由参数 $\theta$ 确定，则 $p(x|c)$ 可以表示为 $p(x|\theta_c)$，设 $D_c$ 表示训练集 $D$ 中 $c$ 类样本的集合，则  

$$p(D_c|\theta_c)=\prod_{x\in D_c}p(x|\theta_c)$$

两边取对数，极大化似然函数。设 $p(x|\theta_c)$ 服从正态分布，我们可以对 $p(x|\theta_c)$ 进行参数估计。若 $p(x|\theta_c)$ 不服从正态分布，那么估计结果是有偏的，为了解决这个问题，提出了下面的朴素贝叶斯模型。

## 朴素贝叶斯模型

朴素贝叶斯模型采用了“属性条件独立性假设”，即所有属性间相互独立。  

$$p(y_i|x)=\frac{p(x|y_i)p(y_i)}{p(x)}=p(x_1|y_i)p(x_2|y_i)...p(x_n|y_i)p(y_i)$$

以下图数据为例，求 $x=(2,S)^T$ 的类标记 $Y$。  
<img align="left" src="img/朴素贝叶斯举例.png" width="150px">  




$P(Y=1)=\frac{9}{15},P(Y=-1)=\frac{6}{15}$  

$P(X_1=1|Y=1)=\frac{2}{9},P(X_1=2|Y=1)=\frac{3}{9},P(X_1=3|Y=1)=\frac{4}{9}$  

$P(X_2=S|Y=1)=\frac{1}{9},P(X_2=M|Y=1)=\frac{4}{9},P(X_2=L|Y=1)=\frac{4}{9}$  

$P(X_1=1|Y=-1)=\frac{3}{6},P(X_1=2|Y=-1)=\frac{2}{6},P(X_1=3|Y=-1)=\frac{1}{6}$  

$P(X_2=S|Y=-1)=\frac{3}{6},P(X_2=M|Y=-1)=\frac{2}{6},P(X_2=L|Y=-1)=\frac{1}{6}$  

对给定的 $x=(2,S)^T$ 计算：  

$P(X_1=2|Y=1)P(X_2=S|Y=1)P(Y=1)=\frac{9}{15}\times\frac{3}{9}\times\frac{1}{9}=\frac{1}{45}$  

$P(X_1=2|Y=-1)P(X_2=S|Y=-1)P(Y=-1)=\frac{6}{15}\times\frac{2}{6}\times\frac{3}{6}=\frac{1}{15}$  

因此其类标记 $Y=-1$

# 朴素贝叶斯代码实现

In [1]:
import numpy as np

def loaddata():
    X = np.array([[1,'S'],[1,'M'],[1,'M'],[1,'S'],
         [1, 'S'], [2, 'S'], [2, 'M'], [2, 'M'],
         [2, 'L'], [2, 'L'], [3, 'L'], [3, 'M'],
         [3, 'M'], [3, 'L'], [3, 'L']])
    y = np.array([-1,-1,1,1,-1,-1,-1,1,1,1,1,1,1,1,-1])
    return X, y

**1.训练，计算各个概率值**

$$p(y_i|x)=p(x_1|y_i)p(x_2|y_i)...p(x_n|y_i)p(y_i)$$  

$P(Y=1)=\frac{9}{15},P(Y=-1)=\frac{6}{15}$  
$P(X_1=1|Y=1)=\frac{2}{9},P(X_1=2|Y=1)=\frac{3}{9},P(X_1=3|Y=1)=\frac{4}{9}$  
$P(X_2=S|Y=1)=\frac{1}{9},P(X_2=M|Y=1)=\frac{4}{9},P(X_2=L|Y=1)=\frac{4}{9}$  
$P(X_1=1|Y=-1)=\frac{3}{6},P(X_1=2|Y=-1)=\frac{2}{6},P(X_1=3|Y=-1)=\frac{1}{6}$  
$P(X_2=S|Y=-1)=\frac{3}{6},P(X_2=M|Y=-1)=\frac{2}{6},P(X_2=L|Y=-1)=\frac{1}{6}$  

In [2]:
def Train(trainset,train_labels):
    m = trainset.shape[0]       # 数据量
    n = trainset.shape[1]       # 特征数
    
    # 先验概率，key是类别值，value是类别的概率值的分子，例如 P(Y=1)表示为{1:9},P(Y=-1)表示为{-1,6}
    prior_prob = {}        
    
    # 条件概率，key：类别，特征，特征值，value是概率值的分子，例如 P(X1=1|Y=1)的key为1，0，1，value为2；P(X2=M|Y=-1)的key为-1，1，M，value为2
    conditional_prob = {}     
    
    # 类别的可能取值
    labels = set(train_labels)
    
    # 计算先验概率（此时没有除以总数量m）
    for label in labels:
        prior_prob[label] = len(train_labels[train_labels==label])
    
    # 计算条件概率
    for i in range(m):
        for j in range(n):
            key = str(train_labels[i])+','+str(j)+','+str(trainset[i][j])
            if key not in conditional_prob.keys():
                conditional_prob[key] = 0
            conditional_prob[key] += 1
            
    # 计算最终的条件概率
    conditional_prob_final = {}       
    for key in conditional_prob:
        y = int(key.split(',')[0])
        conditional_prob_final[key] = conditional_prob[key] / prior_prob[y]
        
    # 计算最终的先验概率
    prior_prob_final = {}
    for label in labels:
        prior_prob_final[label] = prior_prob[label] / m
        
    return prior_prob_final,conditional_prob_final,labels

In [3]:
X,y = loaddata()
prior_prob,conditional_prob,train_labels = Train(X,y)
print("prior_prob=",prior_prob)
print("conditional_prob=",conditional_prob)

prior_prob= {1: 0.6, -1: 0.4}
conditional_prob= {'-1,0,1': 0.5, '-1,1,S': 0.5, '-1,1,M': 0.3333333333333333, '1,0,1': 0.2222222222222222, '1,1,M': 0.4444444444444444, '1,1,S': 0.1111111111111111, '-1,0,2': 0.3333333333333333, '1,0,2': 0.3333333333333333, '1,1,L': 0.4444444444444444, '1,0,3': 0.4444444444444444, '-1,0,3': 0.16666666666666666, '-1,1,L': 0.16666666666666666}


**2.预测**

In [4]:
def predict(data):
    result = {}
    for label in train_labels:
        y_hat = prior_prob[label]
        for i in range(len(data)):
            key = str(label)+','+str(i)+','+str(data[i])
            y_hat *= conditional_prob[key]
        result[label] = y_hat
    print(result)
    return sorted(result.items(),key=lambda x:x[1],reverse=True)[0][0]

In [5]:
y_hat = predict([2,'S'])
print('y_hat=',y_hat)

{1: 0.02222222222222222, -1: 0.06666666666666667}
y_hat= -1


# 拉普拉斯修正

为防止某个条件概率为0，造成最终乘积为0的情况，修正方法如下：  

$$p(y)=\frac{\left\vert D_y \right\vert +1}{\left\vert D \right\vert +N},其中N为训练集D中可能的类别数$$

$$p(x_i|y)=\frac{\left\vert D_{y,x_i} \right\vert +1}{\left\vert D_y \right\vert +N_i},其中N_i表示第i个属性可能的取值数$$

In [6]:
def laplaceTrain(trainset,train_labels):
    m = trainset.shape[0]       # 数据量
    n = trainset.shape[1]       # 特征数
    
    # 先验概率，key是类别值，value是类别的概率值的分子，例如 P(Y=1)表示为{1:9},P(Y=-1)表示为{-1,6}
    prior_prob = {}        
    
    # 条件概率，key：类别，特征，特征值，value是概率值的分子，例如 P(X1=1|Y=1)的key为1，0，1，value为2；P(X2=M|Y=-1)的key为-1，1，M，value为2
    conditional_prob = {}     
    
    # 类别的可能取值
    labels = set(train_labels)
    # 每个特征的取值个数
    feature_counts = {}
    for i in range(n):
        feature_counts[i] = len(np.unique(trainset[:,i]))
    
    # 计算先验概率（此时没有除以总数量m）
    for label in labels:
        prior_prob[label] = len(train_labels[train_labels==label])+1       # 分子+1
    
    # 计算条件概率
    for i in range(m):
        for j in range(n):
            key = str(train_labels[i])+','+str(j)+','+str(trainset[i][j])
            if key not in conditional_prob.keys():
                conditional_prob[key] = 1                                  # 分子+1
            conditional_prob[key] += 1
            
    # 计算最终的条件概率
    conditional_prob_final = {}       
    for key in conditional_prob:
        y = int(key.split(',')[0])
        i = int(key.split(',')[1])
        conditional_prob_final[key] = conditional_prob[key] / (prior_prob[y]+feature_counts[i])    # 分母+N
        
    # 计算最终的先验概率
    prior_prob_final = {}
    for label in labels:
        prior_prob_final[label] = prior_prob[label] / (m+len(labels))     # 分母+N
        
    return prior_prob_final,conditional_prob_final,labels

In [7]:
prior_prob,conditional_prob,train_labels = laplaceTrain(X,y)
print("laplace_prior_prob=",prior_prob)
print("laplace_conditional_prob=",conditional_prob)

laplace_prior_prob= {1: 0.5882352941176471, -1: 0.4117647058823529}
laplace_conditional_prob= {'-1,0,1': 0.4, '-1,1,S': 0.4, '-1,1,M': 0.3, '1,0,1': 0.23076923076923078, '1,1,M': 0.38461538461538464, '1,1,S': 0.15384615384615385, '-1,0,2': 0.3, '1,0,2': 0.3076923076923077, '1,1,L': 0.38461538461538464, '1,0,3': 0.38461538461538464, '-1,0,3': 0.2, '-1,1,L': 0.2}


In [8]:
y_hat = predict([2,'S'])
print('y_hat=',y_hat)

{1: 0.02784545770971111, -1: 0.04941176470588235}
y_hat= -1


# 朴素贝叶斯处理连续型数据

假定 $p(x_i|y)\sim N(\mu_{y,i},\sigma^2_{y,i})$，其中 $\mu_{y,i}$ 和 $\sigma^2_{y,i}$ 分别是第 $y$ 类样本在第 $i$ 个属性上取值的均值和方差。  
$$p(x_i|y)=\frac{1}{\sqrt{2\pi}\sigma_{y,i}}\exp(-\frac{(x_i-\mu_{y,i})^2}{2\sigma^2_{y,i}})$$

<img align="left" src="img/朴素贝叶斯连续型数据.png" width="200px">

# Sklearn实现朴素贝叶斯

In [9]:
from sklearn import naive_bayes as nb
from sklearn.preprocessing import LabelEncoder

需要把字符类型的特征转换为数字类型，即对字符类型进行编码

In [10]:
X[:,1] = LabelEncoder().fit_transform(X[:,1])
X = X.astype(int)
X

array([[1, 2],
       [1, 1],
       [1, 1],
       [1, 2],
       [1, 2],
       [2, 2],
       [2, 1],
       [2, 1],
       [2, 0],
       [2, 0],
       [3, 0],
       [3, 1],
       [3, 1],
       [3, 0],
       [3, 0]])

调用朴素贝叶斯模型并fit数据

In [11]:
model = nb.MultinomialNB()
model.fit(X,y)

MultinomialNB()

对[2,'S']进行预测，同样S要转换成2

In [12]:
print(model.predict([[2,2]]))

[-1]


## 使用朴素贝叶斯对鸢尾花数据进行分类

In [13]:
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris

iris = load_iris()
X = iris.data
y = iris.target

model.fit(X,y)

y_hat = model.predict(X)
print(accuracy_score(y_hat,y))

0.9533333333333334


# 案例：垃圾邮件识别

## 实现原理

1. 对训练数据的处理：读取文件，分词。0表示非垃圾邮件，1表示垃圾邮件。  

<img align="center" src="img/垃圾邮件识别1.png" width="450px">  


2. 获取词汇列表：取出所有邮件中不重复的单词并编码。第 i 行邮件出现的单词位置标为1。  

<img align="center" src="img/垃圾邮件识别2.png" width="800px">  


3. 把训练数据转换为向量（以第一条为例）  

<img align="center" src="img/垃圾邮件识别3.png" width="700px">  


4. 利用朴素贝叶斯求解

## 代码实现

In [14]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.metrics import accuracy_score
from sklearn import naive_bayes as nb
from sklearn.model_selection import train_test_split
import scipy
from scipy import io

**获取词汇列表**

In [15]:
def createVocabList(dataSet):
    vocabSet = set([])
    for document in dataSet:
        # 词汇表和set(document)取并集
        vocabSet = vocabSet | set(document)
        
    # 返回一个经过自然排序的词汇表
    return sorted(list(vocabSet))

In [16]:
dataset = [['i','love','you'],
           ['he','love','you']]
vocablist = createVocabList(dataset)
print(vocablist)

['he', 'i', 'love', 'you']


**词集模型（只要出现就标1）**

In [17]:
def setOfWord2Vec(vocabList,inputSet):
    # 初始化向量，其长度与词汇表长度一致
    returnVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1
    return returnVec

In [18]:
print(setOfWord2Vec(vocablist,['love','you','you']))

[0, 0, 1, 1]


**词袋模型（出现几次就标几）**

In [19]:
def bagOfWord2Vec(vocabList,inputSet):
    # 初始化向量，其长度与词汇表长度一致
    returnVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] += 1
    return returnVec

In [20]:
print(bagOfWord2Vec(vocablist,['love','you','you']))

[0, 0, 1, 2]


**对邮件内容的预处理（此处简单处理一下，实际情况复杂得多）**

In [21]:
def textParse(bigStr):
    import re
    listOfTokens = re.split(r'\W+',bigStr)
    return [tok.lower() for tok in listOfTokens if len(tok)>2]

In [22]:
textParse("I Love You")

['love', 'you']

**加载数据**

In [23]:
def loaddata(num):     # num为训练数据中的垃圾邮件数量
    docList = []
    classList = []

    for i in range(1, num+1):
        wordList = textParse(open('data/email/spam/%d.txt' % i).read())
        docList.append(wordList)
        classList.append(1)
        
        wordList = textParse(open('data/email/ham/%d.txt' % i).read())
        docList.append(wordList)
        classList.append(0)
        
    vocabList = createVocabList(docList)
    
    X = []
    for docIndex in range(len(docList)):
        X.append(bagOfWord2Vec(vocabList, docList[docIndex]))
        # X.append(setOfWord2Vec(vocabList, docList[docIndex]))
        
    return X, classList, vocabList

In [24]:
X,y,vocablist = loaddata(25)

In [25]:
len(vocablist)

694

**利用朴素贝叶斯模型求解**

In [26]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)
model = nb.MultinomialNB()
model.fit(X_train,y_train)
y_hat = model.predict(X_test)
accuracy_score(y_test,y_hat)

1.0